In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

In [ ]:
def redimensionar_imagem(imagem, max_width=800, max_height=600):
    """
    Redimensiona a imagem para otimizar o processamento
    """
    height, width = imagem.shape[:2]

    # Calcula o fator de redimensionamento
    scale_width = max_width / width
    scale_height = max_height / height
    scale = min(scale_width, scale_height, 1.0)  # Não aumenta a imagem

    if scale < 1.0:
        new_width = int(width * scale)
        new_height = int(height * scale)
        return cv2.resize(imagem, (new_width, new_height), interpolation=cv2.INTER_AREA)

    return imagem

def converter_para_cinza(imagem_colorida):
    """
    Converte imagem colorida para níveis de cinza usando a fórmula padrão
    Cinza = 0.299*R + 0.587*G + 0.114*B
    """
    if len(imagem_colorida.shape) == 3:
        # Converte BGR para RGB (OpenCV usa BGR por padrão)
        imagem_rgb = cv2.cvtColor(imagem_colorida, cv2.COLOR_BGR2RGB)

        # Aplica a fórmula de conversão para cinza
        imagem_cinza = np.dot(imagem_rgb[...,:3], [0.299, 0.587, 0.114])
        return imagem_cinza.astype(np.uint8)
    else:
        return imagem_colorida

def binarizar_imagem(imagem_cinza, threshold=128):
    """
    Converte imagem em cinza para binária (preto e branco)
    Pixels >= threshold ficam brancos (255)
    Pixels < threshold ficam pretos (0)
    """
    imagem_binaria = np.where(imagem_cinza >= threshold, 255, 0)
    return imagem_binaria.astype(np.uint8)

def processar_imagem(caminho_imagem, threshold=128, salvar_resultados=True):
    """
    Função principal que processa a imagem completa
    """
    try:
        # Carrega a imagem
        imagem_original = cv2.imread(caminho_imagem)
        if imagem_original is None:
            raise ValueError("Não foi possível carregar a imagem. Verifique o caminho.")

        # Redimensiona para otimizar processamento
        imagem_original = redimensionar_imagem(imagem_original)

        # Converte BGR para RGB para visualização correta
        imagem_original_rgb = cv2.cvtColor(imagem_original, cv2.COLOR_BGR2RGB)

        # Converte para cinza
        imagem_cinza = converter_para_cinza(imagem_original)

        # Binariza a imagem
        imagem_binaria = binarizar_imagem(imagem_cinza, threshold)

        # Visualiza os resultados
        fig, axes = plt.subplots(1, 3, figsize=(15, 5))

        # Imagem original
        axes[0].imshow(imagem_original_rgb)
        axes[0].set_title('Imagem Original')
        axes[0].axis('off')

        # Imagem em cinza
        axes[1].imshow(imagem_cinza, cmap='gray')
        axes[1].set_title('Imagem em Cinza')
        axes[1].axis('off')

        # Imagem binária
        axes[2].imshow(imagem_binaria, cmap='gray')
        axes[2].set_title(f'Imagem Binária (threshold={threshold})')
        axes[2].axis('off')

        plt.tight_layout()
        plt.show()

        # Salva os resultados se solicitado
        if salvar_resultados:
            nome_base = caminho_imagem.split('.')[0]

            # Salva imagem em cinza
            cv2.imwrite(f'{nome_base}_cinza.jpg', imagem_cinza)

            # Salva imagem binária
            cv2.imwrite(f'{nome_base}_binaria.jpg', imagem_binaria)

            print(f"Imagens salvas: {nome_base}_cinza.jpg e {nome_base}_binaria.jpg")

        return imagem_original_rgb, imagem_cinza, imagem_binaria

    except Exception as e:
        print(f"Erro ao processar imagem: {e}")
        return None, None, None

def encontrar_melhor_threshold(imagem_cinza):
    """
    Encontra o melhor threshold usando o método de Otsu
    """
    # Método de Otsu para encontrar threshold ótimo
    threshold_otsu, _ = cv2.threshold(imagem_cinza, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    return int(threshold_otsu)

In [ ]:
# Exemplo de uso
if __name__ == "__main__":
    # Substitua pelo caminho da sua imagem
    caminho_da_imagem = "/content/Panda.jpg"

    print("=== Conversor de Imagem: Colorida → Cinza → Binária ===\n")

    # Processamento com threshold padrão (128)
    print("1. Processando com threshold padrão (128):")
    original, cinza, binaria = processar_imagem(caminho_da_imagem, threshold=128)

    if cinza is not None:
        # Encontra o melhor threshold usando Otsu
        melhor_threshold = encontrar_melhor_threshold(cinza)
        print(f"\n2. Threshold sugerido pelo método de Otsu: {melhor_threshold}")

        # Processamento com threshold otimizado
        print("3. Processando com threshold otimizado:")
        processar_imagem(caminho_da_imagem, threshold=melhor_threshold)

        # Mostra estatísticas das imagens
        print("\n=== Estatísticas ===")
        print(f"Dimensões da imagem: {cinza.shape}")
        print(f"Valor mínimo (cinza): {np.min(cinza)}")
        print(f"Valor máximo (cinza): {np.max(cinza)}")
        print(f"Valor médio (cinza): {np.mean(cinza):.1f}")

        # Conta pixels pretos e brancos na imagem binária
        pixels_pretos = np.sum(binaria == 0)
        pixels_brancos = np.sum(binaria == 255)
        total_pixels = binaria.size

        print(f"\nDistribuição na imagem binária:")
        print(f"Pixels pretos: {pixels_pretos} ({pixels_pretos/total_pixels*100:.1f}%)")
        print(f"Pixels brancos: {pixels_brancos} ({pixels_brancos/total_pixels*100:.1f}%)")

# Função adicional para testar diferentes thresholds
def testar_multiplos_thresholds(caminho_imagem, thresholds=[64, 128, 192]):
    """
    Testa diferentes valores de threshold para comparação
    """
    imagem_original = cv2.imread(caminho_imagem)
    if imagem_original is None:
        print("Erro: Não foi possível carregar a imagem")
        return

    imagem_original = redimensionar_imagem(imagem_original)
    imagem_cinza = converter_para_cinza(imagem_original)

    fig, axes = plt.subplots(2, len(thresholds), figsize=(15, 8))

    for i, threshold in enumerate(thresholds):
        imagem_binaria = binarizar_imagem(imagem_cinza, threshold)

        # Primeira linha: imagem em cinza
        if len(thresholds) > 1:
            axes[0, i].imshow(imagem_cinza, cmap='gray')
            axes[0, i].set_title(f'Imagem em Cinza')
            axes[0, i].axis('off')

            # Segunda linha: imagem binária
            axes[1, i].imshow(imagem_binaria, cmap='gray')
            axes[1, i].set_title(f'Threshold = {threshold}')
            axes[1, i].axis('off')
        else:
            axes[0].imshow(imagem_cinza, cmap='gray')
            axes[0].set_title(f'Imagem em Cinza')
            axes[0].axis('off')

            axes[1].imshow(imagem_binaria, cmap='gray')
            axes[1].set_title(f'Threshold = {threshold}')
            axes[1].axis('off')

    plt.tight_layout()
    plt.show()